# AutoGluon Quick Start

#### source 
- https://autogluon.mxnet.io/

#### 참고
- 한글 동영상 - https://youtu.be/xnimaVNTWfc
- aws 블로그  - [https://aws.amazon.com/ko/blogs/opensource/machine-learning-with-autogluon-an-open-source-automl-library/](https://aws.amazon.com/ko/blogs/opensource/machine-learning-with-autogluon-an-open-source-automl-library/)
- 블로그의 샘플코드 - [https://github.com/shashankprasanna/autogluon-demos/blob/master/otto-kaggle-example.ipynb](https://github.com/shashankprasanna/autogluon-demos/blob/master/otto-kaggle-example.ipynb)
- 데모 - [https://github.com/shashankprasanna/autogluon-demos](https://github.com/shashankprasanna/autogluon-demos)
- 소스 - [https://github.com/awslabs/autogluon](https://github.com/awslabs/autogluon)
- paper - [https://arxiv.org/abs/2003.06505](https://arxiv.org/abs/2003.06505) [https://arxiv.org/pdf/2003.06505.pdf](https://arxiv.org/pdf/2003.06505.pdf)


In [16]:
!pip install --upgrade "mxnet<2.0.0"
!pip install autogluon

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


### AutoGluon API

```python
from autogluon import TabularPrediction as task

# 1) Dataset() - pandas와 유사
data = task.Dataset(DATASET_PATH)

# 2) fit() - 몇가지 모델이 만들어져서 디스크에 저장됨
predictor = task.fit(data_train, label=LABEL_COLUMN_NAME)

# 3) predict()
prediction = predictor.predict(new_data)
```

In [11]:
import autogluon as ag
# from autogluon import TabularPrediction as task
from autogluon.tabular import TabularDataset, TabularPredictor

### 1) Dataset()

In [13]:
train_data = TabularDataset(data='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = train_data.head(500) # subsample 500 data points for faster demo
# print(train_data.head())

In [14]:
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K


In [15]:
label_column = 'class'
train_data.groupby(label_column).count()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
class,,,,,,,,,,,,,,
<=50K,394,394,394,394,394,394,394,394,394,394,394,394,394,394
>50K,106,106,106,106,106,106,106,106,106,106,106,106,106,106


### 2) fit()

In [18]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label_column, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 4 threads.
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:     

In [19]:
test_data = TabularDataset(data='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(columns=[label_column])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


### 3) predict()

In [20]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8220902855972976
Evaluations on test data:
{
    "accuracy": 0.8220902855972976,
    "balanced_accuracy": 0.7141181169953735,
    "mcc": 0.4721510555794738,
    "f1": 0.5756835937499999,
    "precision": 0.6631046119235096,
    "recall": 0.50862812769629
}


Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [21]:
leaderboard = predictor.leaderboard(test_data, silent=True)
leaderboard.head()

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.834988,0.84,0.026166,0.010691,0.636528,0.026166,0.010691,0.636528,1,True,7
1,LightGBMXT,0.822090,0.87,0.130190,0.020073,0.779172,0.130190,0.020073,0.779172,1,True,3
2,WeightedEnsemble_L2,0.822090,0.87,0.133080,0.020805,1.260792,0.002890,0.000733,0.481620,2,True,14
3,RandomForestEntr,0.821578,0.83,0.222880,0.110375,0.667304,0.222880,0.110375,0.667304,1,True,6
4,RandomForestGini,0.821578,0.83,0.223232,0.110590,0.691156,0.223232,0.110590,0.691156,1,True,5


In [29]:
predictor.predict(test_data_nolab, model='LightGBMXT')

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

## Behind the fit function

#### Data preprocessing
1. Classification 문제인지 regression 문제인지 결정
2. Feature를 numeric, categorical, text, or date/time 타입으로 분류하고 전처리
    - categorical 변수이지만 반복되지 않는 컬럼은 삭제함 (예: user_id 컬럼)
    - text 컬럼은 n-gram 으로, date/time은 적절한 숫자 타입으로 변환 등
    - missing 데이터는 unkown으로 채움
3. Training set/ validation set 분리

  
#### Model fitting
- random forest처럼 범용적인 알고리즘부터 try하고 knn같은것은 나중에 시도함
- 제약사항이 없이 가장 정확한 모델리턴 or 비용/시간 제한을 주고 최고 정확성 모델 리턴 
- 앙상블 and multi-layer stacking을 이용한 결합
- 하이퍼파라미터 최적화


#### Algorithms
- 알고리즘 구성 (초기구성)
    1. [Random Forests](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
    2. [Extremely Randomized trees](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)
    3. [k-nearest neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
    4. [LightGBM boosted trees](https://lightgbm.readthedocs.io/en/latest/)
    5. [CatBoost boosted trees](https://github.com/catboost/catboost)
    6. AutoGluon-Tabular deep neural networks

  
#### AutoGluon DNN 네트워크 아키텍처 
<img src="https://d2908q01vomqb2.cloudfront.net/ca3512f4dfa95a03169c5a670a4c91a19b3077b4/2020/03/30/autogluon_f2.png" width='500' >  

- embedding layer가 있는 것이 특징

#### Ensembles and multi-layer stacking  
<img src="https://d2908q01vomqb2.cloudfront.net/ca3512f4dfa95a03169c5a670a4c91a19b3077b4/2020/03/30/autogluon_f3-1024x744.png" width='430' >

- Base layer: n개의 모델로 개별 학습
- Concat layer: 각 모델의 예측 결과를 input 값과 함께 합치기
- Stacker layer: concat layer의 결과를 입력으로 다수의 stacker 모델을 이용하여 학습 실행. 이 때 base layer의 모델들을 재사용함(하이퍼파라미터 포함
- Weighting layer: stacker layer의 결과는 새로운 ensamble모델을 이용하여 validation set의 정확도를 높일도록 다시 합쳐짐